In [1]:
import os
import sys

from tqdm import tqdm
import pandas as pd
import numpy as np
import scipy.stats as st

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append(os.path.realpath(os.path.abspath('..')))

from iDrink.iDrinkUtilities import get_drivepath, get_title_measure_name, get_unit, get_cad, get_setting_axis_name, get_measure_short_name

In [7]:
drive = get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_stat_cat = os.path.join(root_stat, '02_categorical')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots_cont = os.path.join(dir_stat_cont, '02_plots')
dir_plots_cat = os.path.join(dir_stat_cat, '02_plots')

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')


csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')
df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']  # Becaus bad calibration reprojection error

murphy_measures = ['PeakVelocity_mms', 'elbowVelocity', 'tTopeakV_s', 'tToFirstpeakV_s',
       'tTopeakV_rel', 'tToFirstpeakV_rel', 'NumberMovementUnits',
       'InterjointCoordination', 'trunkDisplacementMM', 'trunkDisplacementDEG',
       'ShoulderFlexionReaching', 'ElbowExtension', 'shoulderAbduction',
       'shoulderFlexionDrinking']

ignore_id_p = ['P11', 'P15', 'P19']
idx_s_singlecam_full = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']
idx_s_singlecam = ['S017', 'S018']
idx_s_multicam = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016']
idx_s_multicam_reduced = ['S001', 'S002']
idx_s_reduced = idx_s_multicam_reduced + idx_s_singlecam
idx_s_full = idx_s_multicam + idx_s_singlecam

reduced_analysis = True # If True, only reduced set of subjects will be analyzed

# Bland Altman Tables
## Kinematic Measures

In [3]:
df_murphy = df_murphy[df_murphy['id_p'].isin(ignore_id_p) == False]
df_murphy.drop(columns=['valid', 'ReachingStart',
       'ForwardStart', 'DrinkingStart', 'BackStart', 'ReturningStart',
       'RestStart', 'TotalMovementTime'], inplace=True)

In [4]:
df_altman = pd.DataFrame(columns=['id_s', 'id_p', 'measure', 'mean_err', 'lower_loa', 'upper_loa'])

In [5]:
cols_error = ['PeakVelocity_mms', 'elbowVelocity', 'tTopeakV_s',
              'tToFirstpeakV_s', 'tTopeakV_rel', 'tToFirstpeakV_rel', 'NumberMovementUnits',
              'InterjointCoordination', 'trunkDisplacementMM', 'trunkDisplacementDEG', 'ShoulderFlexionReaching',
              'ElbowExtension', 'shoulderAbduction', 'shoulderFlexionDrinking']
df_murphy_full = df_murphy.copy()

In [8]:
from scipy.stats import zscore

df_murphy_omc_s = df_murphy[df_murphy['id_s'] == 'S15133']
df_murphy_mmc = df_murphy[df_murphy['id_s'] != 'S15133']
cols_error = ['PeakVelocity_mms', 'elbowVelocity', 'tTopeakV_s',
              'tToFirstpeakV_s', 'tTopeakV_rel', 'tToFirstpeakV_rel', 'NumberMovementUnits',
              'InterjointCoordination', 'trunkDisplacementMM', 'trunkDisplacementDEG', 'ShoulderFlexionReaching',
              'ElbowExtension', 'shoulderAbduction', 'shoulderFlexionDrinking']

if reduced_analysis:
    df_murphy_mmc = df_murphy_mmc[df_murphy_mmc['id_s'].isin(idx_s_full)]
    df_murphy_mmc = df_murphy_mmc[~df_murphy_mmc['id_p'].isin(ignore_id_p)]
    df_murphy_omc_s = df_murphy_omc_s[~df_murphy_omc_s['id_p'].isin(ignore_id_p)]

df_murphy_mmc = df_murphy_mmc[(np.abs(zscore(df_murphy_mmc[cols_error]).fillna(0)) < 3).all(axis=1)].reindex()
df_murphy_omc_s = df_murphy_omc_s[(np.abs(zscore(df_murphy_omc_s[cols_error])) < 3).all(axis=1)].reindex()

In [11]:
idx_s = df_murphy_mmc['id_s'].unique()
idx_p = set(df_murphy_omc_s['id_p'].unique()).intersection(set(df_murphy_mmc['id_p'].unique()))

In [9]:
make_plots = False

In [12]:
df_altman = pd.DataFrame(columns=['id_s', 'id_p', 'measure', 'mean_err', 'lower_loa', 'upper_loa'])
total = len(idx_s) * len(murphy_measures)

progbar = tqdm(total=total, desc='Processing')
for measure in murphy_measures:
    
    measure_name = get_title_measure_name(measure)
    unit = get_unit(measure)
    colors = px.colors.qualitative.Plotly
    
    for id_s in idx_s:
        
        progbar.set_description(f'Processing {id_s} \t {measure}')
        
        df_murphy_mmc_s = df_murphy_mmc[df_murphy_mmc['id_s'] == id_s]
        i = 0
        
        if make_plots:
            fig_val_val = go.Figure()
        max_val = 0
        
        df_omc_fit = None
        df_mmc_fit = None
        
        id_s_name = get_setting_axis_name(id_s)
        
        for id_p in idx_p:
            df_murphy_omc_p = df_murphy_omc_s[df_murphy_omc_s['id_p'] == id_p]
            df_murphy_mmc_p = df_murphy_mmc_s[df_murphy_mmc_s['id_p'] == id_p]
            
            idx_t = set(df_murphy_omc_p['id_t'].unique()).intersection(set(df_murphy_mmc_p['id_t'].unique()))
        
            df_murphy_omc_p = df_murphy_omc_p[df_murphy_omc_p['id_t'].isin(idx_t)].sort_values(by='id_t')
            df_murphy_mmc_p = df_murphy_mmc_p[df_murphy_mmc_p['id_t'].isin(idx_t)].sort_values(by='id_t')
            
            if df_omc_fit is None:
                df_omc_fit = df_murphy_omc_p
                df_mmc_fit = df_murphy_mmc_p
            else:
                df_omc_fit = pd.concat([df_omc_fit, df_murphy_omc_p], ignore_index=True)
                df_mmc_fit = pd.concat([df_mmc_fit, df_murphy_mmc_p], ignore_index=True)
            
            # get values and add to DataFrame
            means = np.nanmean([df_murphy_omc_p[measure].values, df_murphy_mmc_p[measure].values], axis=0)
            
            diff = df_murphy_mmc_p[measure].values - df_murphy_omc_p[measure].values
            mean_err = np.nanmean(diff)
            std = np.nanstd(diff, ddof=1)
            unit = get_unit(measure)
            
            # Confidence level
            confidence = 0.95
            loas = st.norm.interval(confidence, mean_err, std)
            lower_loa = loas[0]
            upper_loa = loas[1]
            
            std_diff = np.std(diff)
            sd = 1.96
            upper_loa = mean_err + sd * std_diff
            lower_loa = mean_err - sd * std_diff
            
            df_altman = pd.concat([df_altman, pd.DataFrame({'id_s': id_s, 'id_p': id_p, 'measure': measure, 'mean_err': mean_err, 'lower_loa': lower_loa, 'upper_loa': upper_loa}, index = [0])], ignore_index=True)
            
            # Make Value-Value Plot
            
            if make_plots:
                try:
                    new_max = max(np.nanmax(df_murphy_mmc_p[measure].values), np.nanmax(df_murphy_omc_p[measure].values))
                    if max_val < new_max:
                        max_val = new_max
                except:
                    pass
                
            
                fig_val_val.add_trace(go.Scatter(x=df_murphy_mmc_p[measure].values, y=df_murphy_omc_p[measure].values, mode='markers', name=f'{id_p}', marker=dict(color=colors[i], size=10)))
                fig_val_val.update_layout(title=f'{measure_name} for {id_s_name}', xaxis_title=f'MMC {measure_name} [{unit}]', yaxis_title=f'OMC {measure_name} [{unit}]', template='plotly')
                
            i+=1
        
        if make_plots:    
            fig_val_val.add_trace(go.Scatter(x=[0, max_val], y=[0, max_val], mode='lines', name='Line of Equality', line=dict(color='grey', width=2, dash='dash')))
            
            try:
                regress = st.linregress(x=df_mmc_fit[measure].values, y=df_omc_fit[measure].values)
                fig_val_val.add_trace(go.Scatter(x=df_mmc_fit[measure].values, y=regress.intercept + regress.slope * df_mmc_fit[measure].values, mode='lines', name='Regression Line', line=dict(color='red', width=2)))
                
                fig_val_val.update_layout(template='plotly', height = 1000, width = 1000)
            
                fig_val_val.add_annotation(
                    x=max_val,  # Set the annotation near the maximum x value
                    y=0,  # Set the annotation near the minimum y value
                    text=f"Slope: {regress.slope:.2f}",  # Display the slope with 2 decimal places
                    showarrow=False,  # No arrow needed
                    font=dict(size=12),  # Adjust font size
                    xanchor="right",  # Align to the right
                    yanchor="bottom"  # Align to the bottom
                    )
            except:
                pass
        
        # If nregress is None, do not write the image
        
        progbar.update(1)
progbar.close()

df_altman.dropna()

Processing S005 	 PeakVelocity_mms:   1%|▏         | 2/154 [00:00<00:05, 27.94it/s]C:\Users\johan\AppData\Local\Temp\ipykernel_14492\1263038907.py:47: RuntimeWarning: Mean of empty slice
  mean_err = np.nanmean(diff)
C:\Users\johan\anaconda3\envs\iDrink_GUI\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\johan\anaconda3\envs\iDrink_GUI\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\johan\anaconda3\envs\iDrink_GUI\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\johan\anaconda3\envs\iDrink_GUI\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Use

,id_s,id_p,measure,mean_err,lower_loa,upper_loa
0,S001,P241,PeakVelocity_mms,-10.495474,-183.823695,162.832747
1,S001,P252,PeakVelocity_mms,31.269493,-163.559470,226.098456
2,S001,P242,PeakVelocity_mms,87.901803,-5.008004,180.811610
3,S001,P07,PeakVelocity_mms,-47.115979,-166.071727,71.839768
4,S001,P12,PeakVelocity_mms,251.793372,-374.208784,877.795529
...,...,...,...,...,...,...
1221,S007,P251,shoulderFlexionDrinking,-22.521646,-36.567108,-8.476185
1222,S007,P10,shoulderFlexionDrinking,-3.714852,-20.345379,12.915675
1224,S013,P241,shoulderFlexionDrinking,-20.706104,-47.147591,5.735383
1228,S013,P12,shoulderFlexionDrinking,-31.724910,-53.033695,-10.416124


## Make Plot for Powerpoint

In [18]:
df_altman['id_s'].unique()

array(['S001', 'S002', 'S005', 'S006', 'S008', 'S014', 'S016', 'S017',
       'S018', 'S007', 'S013'], dtype=object)

In [13]:
dir_out = r'C:\Users\johan\OneDrive\Dokumente\Studium\LLUI_Praktikum\09 Masterarbeit\02 Präsentationen\Final Presentation'

id_s_for_powerpoint = ['S001', 'S002', 'S017', 'S018']

df_altman_powerpoint = df_altman[df_altman['id_s'].isin(idx_s_full)]

df_altman_powerpoint = df_altman_powerpoint.dropna().groupby(['id_s', 'measure'], as_index=False).median(numeric_only=True)

#df_altman_powerpoint['id_s_name'] = df_altman_powerpoint['id_s'].apply(lambda x: get_setting_axis_name(x))

df_altman_powerpoint.insert(0, 'id_s_name', df_altman_powerpoint['id_s'].apply(lambda x: get_setting_axis_name(x)))
df_altman_powerpoint.drop(columns=['id_s', 'mean_err'], inplace=True)
df_altman_powerpoint = df_altman_powerpoint[df_altman_powerpoint['measure'] != 'trunkDisplacementDEG']

df_altman_powerpoint['CAD'] = df_altman_powerpoint['measure'].apply(lambda x: df_cad[get_measure_short_name(x)].values[0])
df_altman_powerpoint['measure'] = df_altman_powerpoint['measure'].apply(lambda x: get_measure_short_name(x))
df_altman_powerpoint['LoA < CAD'] = df_altman_powerpoint.apply(
    lambda row: 'Yes' if (row['lower_loa'] > -row['CAD']) and (row['upper_loa'] < row['CAD']) else 'No', axis=1)

#round loa to 2 decimal places
df_altman_powerpoint['lower_loa'] = df_altman_powerpoint['lower_loa'].apply(lambda x: round(x, 2))
df_altman_powerpoint['upper_loa'] = df_altman_powerpoint['upper_loa'].apply(lambda x: round(x, 2))

df_altman_powerpoint.rename(columns={'id_s_name': 'Setting', 'measure': 'Measure', 'lower_loa': 'LoA Low', 'upper_loa': 'LoA Up'}, inplace=True)

df_altman_powerpoint.to_csv(os.path.join(dir_out, 'bland_altman_powerpoint.csv'),sep=';', index=False)

In [16]:
df_altman_powerpoint

,Setting,Measure,LoA Low,LoA Up,CAD,LoA < CAD
0,"SimCC, Cams: 1,2,3,4,5",elb_ext,-0.64,18.28,7.82,No
1,"SimCC, Cams: 1,2,3,4,5",interj_coord,-0.21,0.24,0.18,No
2,"SimCC, Cams: 1,2,3,4,5",n_mov_units,-4.12,6.47,2.64,No
3,"SimCC, Cams: 1,2,3,4,5",peak_V,-164.82,193.95,78.96,No
4,"SimCC, Cams: 1,2,3,4,5",arm_flex_reach,-14.62,0.80,4.48,No
...,...,...,...,...,...,...
148,"Single, Cam: 1, unfilt",t_first_PV_rel,-15.56,11.86,8.72,No
149,"Single, Cam: 1, unfilt",t_first_PV,-1.35,0.96,0.08,No
150,"Single, Cam: 1, unfilt",t_PV_rel,-58.91,34.00,5.86,No
151,"Single, Cam: 1, unfilt",t_to_PV,-4.62,2.79,0.13,No


In [17]:
print(df_altman_powerpoint.style.to_latex(label='tab:app:cat:altman',
                                          caption='Bland-Altman Analysis for the different measures and settings',
                                          position_float='centering'))

\begin{table}
\centering
\caption{Bland-Altman Analysis for the different measures and settings}
\label{tab:app:cat:altman}
\begin{tabular}{lllrrrl}
 & Setting & Measure & LoA Low & LoA Up & CAD & LoA < CAD \\
0 & SimCC, Cams: 1,2,3,4,5 & elb_ext & -0.640000 & 18.280000 & 7.820000 & No \\
1 & SimCC, Cams: 1,2,3,4,5 & interj_coord & -0.210000 & 0.240000 & 0.180000 & No \\
2 & SimCC, Cams: 1,2,3,4,5 & n_mov_units & -4.120000 & 6.470000 & 2.640000 & No \\
3 & SimCC, Cams: 1,2,3,4,5 & peak_V & -164.820000 & 193.950000 & 78.960000 & No \\
4 & SimCC, Cams: 1,2,3,4,5 & arm_flex_reach & -14.620000 & 0.800000 & 4.480000 & No \\
5 & SimCC, Cams: 1,2,3,4,5 & peak_V_elb & 0.360000 & 1.670000 & 21.710000 & Yes \\
6 & SimCC, Cams: 1,2,3,4,5 & arm_abd & 7.160000 & 20.870000 & 9.990000 & No \\
7 & SimCC, Cams: 1,2,3,4,5 & arm_flex_drink & -16.200000 & -1.050000 & 6.850000 & No \\
8 & SimCC, Cams: 1,2,3,4,5 & t_first_PV_rel & -7.400000 & 8.400000 & 8.720000 & Yes \\
9 & SimCC, Cams: 1,2,3,4,5 & t_first

In [14]:
#df_altman_powerpoint.pivot(index='id_s_name', columns='measure', values=['lower_loa', 'upper_loa'])

KeyError: 'id_s_name'

In [26]:
df_latex = df_altman.dropna().groupby(['id_s', 'measure'], as_index=False).mean(numeric_only=True)

In [27]:
df_latex

,id_s,measure,mean_err,lower_loa,upper_loa
0,S001,ElbowExtension,7.926831,2.226961,13.626701
1,S001,InterjointCoordination,0.024625,-0.128090,0.177340
2,S001,NumberMovementUnits,0.250850,-4.717989,5.219690
3,S001,PeakVelocity_mms,44.709112,-106.940032,196.358256
4,S001,ShoulderFlexionReaching,-8.549074,-14.321423,-2.776725
5,S001,elbowVelocity,1.076739,0.484208,1.669270
6,S001,shoulderAbduction,14.432615,8.675297,20.189933
7,S001,shoulderFlexionDrinking,-10.741054,-16.932250,-4.549858
8,S001,tToFirstpeakV_rel,0.670699,-7.454709,8.796106
9,S001,tToFirstpeakV_s,0.054677,-0.636082,0.745437


In [28]:
print(df_latex.style.to_latex())

\begin{tabular}{lllrrr}
 & id_s & measure & mean_err & lower_loa & upper_loa \\
0 & S001 & ElbowExtension & 7.926831 & 2.226961 & 13.626701 \\
1 & S001 & InterjointCoordination & 0.024625 & -0.128090 & 0.177340 \\
2 & S001 & NumberMovementUnits & 0.250850 & -4.717989 & 5.219690 \\
3 & S001 & PeakVelocity_mms & 44.709112 & -106.940032 & 196.358256 \\
4 & S001 & ShoulderFlexionReaching & -8.549074 & -14.321423 & -2.776725 \\
5 & S001 & elbowVelocity & 1.076739 & 0.484208 & 1.669270 \\
6 & S001 & shoulderAbduction & 14.432615 & 8.675297 & 20.189933 \\
7 & S001 & shoulderFlexionDrinking & -10.741054 & -16.932250 & -4.549858 \\
8 & S001 & tToFirstpeakV_rel & 0.670699 & -7.454709 & 8.796106 \\
9 & S001 & tToFirstpeakV_s & 0.054677 & -0.636082 & 0.745437 \\
10 & S001 & tTopeakV_rel & 0.870421 & -33.737540 & 35.478382 \\
11 & S001 & tTopeakV_s & 0.051567 & -2.679103 & 2.782237 \\
12 & S001 & trunkDisplacementDEG & 0.286464 & -9.148074 & 9.721002 \\
13 & S001 & trunkDisplacementMM & -1.007706 &

In [24]:
print(df_latex.to_markdown())

|                                     |       mean_err |     lower_loa |     upper_loa |
|:------------------------------------|---------------:|--------------:|--------------:|
| ('S001', 'ElbowExtension')          |    11.2945     |     -1.89686  |    24.4859    |
| ('S001', 'InterjointCoordination')  |     0.0829839  |     -0.221662 |     0.38763   |
| ('S001', 'NumberMovementUnits')     |     0.595074   |     -5.51046  |     6.7006    |
| ('S001', 'PeakVelocity_mms')        |  1856.08       |   -852.636    |  4564.8       |
| ('S001', 'ShoulderFlexionReaching') |   -10.8122     |    -20.5204   |    -1.10406   |
| ('S001', 'elbowVelocity')           |     1.50349    |     -1.13668  |     4.14366   |
| ('S001', 'shoulderAbduction')       |    16.8164     |      8.7052   |    24.9276    |
| ('S001', 'shoulderFlexionDrinking') |   -10.3901     |    -23.9209   |     3.14079   |
| ('S001', 'tToFirstpeakV_rel')       |     0.135468   |    -39.4411   |    39.712     |
| ('S001', 'tToFirstp